**Предисловие:**

**Реализация основана на работе:** ...

**Результаты данной работы представлены в статье (при использовании просьба ссылаться на следующую работу):** ...

**Выборки данных:**
>*    Подписи к изображениям на английском языке из выборки COCO Image Captions
*    Стихи на русском языке с сайта stihi.ru

**Описание работы:**
> В данной работе проведены реализация, обучение и тестирование нейронных сетей для генерации случайных коротких текстов. Используются такие нейронные сети как LSTM и SeqGAN. Проведена оценка качества генерации текста на основе метрики BLEU. Для обучения и тестирования используются следующие выборки данных:
*   Подписи к изображениям на английском языке из выборки COCO Image Captions
*   Стихи на русском языке с сайта stihi.ru




Список всех установленных библиотек и их версии

In [40]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

Подключение к Google Drive

In [41]:
#подключение к гугл диску
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import os

path = '/content/drive/My Drive/SeqGAN/'
os.chdir(path)

Объявление параметров по умолчанию

In [43]:
from __future__ import print_function
from math import ceil
import numpy as np
import sys

import torch
import torch.optim as optim
import torch.nn as nn

import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.init as init

import helpers # перетащить функции в код

DATASET = 'stihi_ru' # название выборки данных список выборок: ['stihi_ru', 'coco_image_captions']
CUDA = True # использование cuda (gpu/tpu): True/False
BATCH_SIZE = 1000 # количество примеров в партии/батче
MLE_TRAIN_EPOCHS = 30 # количество эпох при обучении генератора на основе MLE
DIS_TRAIN_ITERATIONS = 300 # количество итераций при обучении дискриминатора (итерации содержит несколько эпох, на каждой итерации данные для обучения обновляются)
DIS_TRAIN_EPOCHS = 1 # количество эпох в одной итерации обучения дискриминатора
ADV_TRAIN_EPOCHS = 25 # количество эпох при обучении генератора на основе соревновательного обучения / обучения с подкреплением
POS_NEG_SAMPLES = 10000

GEN_EMBEDDING_DIM = 150 # ширина слоя Embedding генератора
GEN_HIDDEN_DIM = 150 # количество нейронов в слоях генератора
DIS_EMBEDDING_DIM = 150 # ширина слоя Embedding дискриминатора
DIS_HIDDEN_DIM = 150 # количество нейронов в слоях дискриминатора

# параметры START_LETTER, VOCAB_SIZE, MAX_SEQ_LEN и FILE_PATHS задаются автоматически ниже
START_LETTER = None #! стартовое слово (слово которое подается первым на нейронную сеть генератор)
MAX_SEQ_LEN = None # длина генерируемого примера
VOCAB_SIZE = None # количество слов в словаре
FILE_PATHS = None # пути к файлам набора данных
CLOSING_WORD = None # заключительное слово / слово заполнитель ставящееся в конце предложения

# выбор набора данных и инициализация соответствующих глобальных параметров
if DATASET == 'stihi_ru': # выборка с русскими стихами с сайта stihi_ru (длина примера 10 слов, размер словаря 5 000 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 10
  VOCAB_SIZE = 5000
  CLOSING_WORD = 4999#4997
  FILE_PATHS = {'train': r'datasets/stihi_ru/stihi_ru_realtrain_cotra.txt', 'test': r'datasets/stihi_ru/stihi_ru_realtest_coco.txt',
                'vocab': r'datasets/stihi_ru/stihi_ru_vocab_cotra.pkl', 'saved_models': r'saved_models/stihi_ru'}
elif DATASET == 'coco_image_captions': # выборка COCO Image Captions (длина примера 20 слов, размер словаря 4980 слов)
  START_LETTER = 0
  MAX_SEQ_LEN = 20
  VOCAB_SIZE = 4838
  CLOSING_WORD = 1814
  FILE_PATHS = {'train': r'datasets/coco_image_captions/coco_image_captions_train.txt', 'test': r'datasets/coco_image_captions/coco_image_captions_test.txt',
                'vocab': r'datasets/coco_image_captions/coco_image_captions_vocab_cotra_test.pkl', 'saved_models': r'saved_models/coco_image_captions'}

Инициализация генераторов случайных чисел.

In [44]:
# инициализация генераторов случайных чисел.
torch.random.manual_seed(50)
np.random.seed(50)
#системный рандом!!!

Объявление класса Генератора

In [45]:
class Generator(nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False):
    super(Generator, self).__init__()
    self.hidden_dim = hidden_dim # количество элементов на скрытом слое
    self.embedding_dim = embedding_dim # размер слоя embedding
    self.max_seq_len = max_seq_len # длина генерируемых примеров
    self.vocab_size = vocab_size # размер словаря использующегося при генерации
    self.gpu = gpu # использование cuda (True/False)
    self.lstm_num_layers = 1 # количество LSTM слоев

    self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя embeddings
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.lstm_num_layers) # объявление LSTM слоев
    self.lstm2out = nn.Linear(hidden_dim, vocab_size) # объявление выходного слоя

  def init_hidden(self, batch_size=1):
    # инициализация состояния LSTM слоев
    h = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim)) 
    c = autograd.Variable(torch.zeros(self.lstm_num_layers, batch_size, self.hidden_dim))

    if self.gpu:
        return h.cuda(), c.cuda()
    else:
        return h, c

  def forward(self, inp, hidden, c):
    """
    Embeds input and applies LSTM one token at a time (seq_len = 1)
    """
    # input dim                                             # batch_size
    emb = self.embeddings(inp)                              # batch_size x embedding_dim
    emb = emb.view(1, -1, self.embedding_dim)               # 1 x batch_size x embedding_dim
    out, (hidden, c) = self.lstm(emb, (hidden, c))                    # 1 x batch_size x hidden_dim (out)
    out = self.lstm2out(out.view(-1, self.hidden_dim))       # batch_size x vocab_size
    out = F.log_softmax(out, dim=1)
    return out, hidden, c

  def sample(self, num_samples, start_letter=0, degree=1):
    """
    Samples the network and returns num_samples samples of length max_seq_len.

    Outputs: samples, hidden
        - samples: num_samples x max_seq_length (a sampled sequence in each row)
    """

    samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

    h, c = self.init_hidden(num_samples)
    inp = autograd.Variable(torch.LongTensor([start_letter]*num_samples))
    if self.gpu:
        samples = samples.cuda()
        inp = inp.cuda()

    for i in range(self.max_seq_len):
        out, h, c = self.forward(inp, h, c)               # out: num_samples x vocab_size
        out = torch.exp(out)**degree
        out = torch.multinomial(out, 1)  # num_samples x 1 (sampling from each row)
        samples[:, i] = out.view(-1).data

        inp = out.view(-1)

    return samples

  def batchNLLLoss(self, inp, target):
    """
    Returns the NLL Loss for predicting target sequence.

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len

        inp should be target with <s> (start letter) prepended
    """

    loss_fn = nn.NLLLoss()
    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)           # seq_len x batch_size
    target = target.permute(1, 0)     # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        loss += loss_fn(out, target[i])

    return loss     # per batch

  def batchPGLoss(self, inp, target, reward):
    """
    Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
    Inspired by the example in http://karpathy.github.io/2016/05/31/rl/

    Inputs: inp, target
        - inp: batch_size x seq_len
        - target: batch_size x seq_len
        - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                  sentence)

        inp should be target with <s> (start letter) prepended
    """

    batch_size, seq_len = inp.size()
    inp = inp.permute(1, 0)          # seq_len x batch_size
    target = target.permute(1, 0)    # seq_len x batch_size
    h, c = self.init_hidden(batch_size)

    loss = 0
    for i in range(seq_len):
        out, h, c = self.forward(inp[i], h, c)
        # TODO: should h be detached from graph (.detach())?
        for j in range(batch_size):
            loss += -out[j][target.data[i][j]]*reward[j]#     # log(P(y_t|Y_1:Y_{t-1})) * Q

    return loss/batch_size

Объявяление класса Дискриминатора

In [46]:
class Discriminator(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.2):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim # размер скрытого слоя
        self.embedding_dim = embedding_dim # ширина слоя embedding
        self.max_seq_len = max_seq_len # длина входного примера
        self.gpu = gpu # использование cuda (True/False)

        self.embeddings = nn.Embedding(vocab_size, embedding_dim) # объявление слоя Embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3, bidirectional=True, dropout=dropout) # объявление LSTM слоев

        self.lstm2hidden = nn.Linear(2*3*hidden_dim, hidden_dim) # объявление скрытого слоя
        # self.lstm2hidden = nn.Linear(max_seq_len*hidden_dim*2, hidden_dim) # объявление скрытого слоя

        self.dropout_linear = nn.Dropout(p=dropout)
        self.hidden2out = nn.Linear(hidden_dim, 1) # объявление выходного слоя

    def init_hidden(self, batch_size):
        # инициализация LSTM слоев
        h = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))
        c = autograd.Variable(torch.zeros(2*3*1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda(), c.cuda()
        else:
            return h, c

    def forward(self, input, hidden, c):
        # input dim                                                # batch_size x seq_len
        # print(input.shape)
        emb = self.embeddings(input)                               # batch_size x seq_len x embedding_dim
        # print(emb.shape)
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim
        # print(emb.shape)
        out_lstm, (hidden, c) = self.lstm(emb, (hidden, c))                          # 4 x batch_size x hidden_dim

        hidden = hidden.permute(1, 0, 2).contiguous()              # batch_size x 4 x hidden_dim
        # hidden = out_lstm
        # print(hidden.shape, hidden.view(-1, self.max_seq_len*self.hidden_dim).shape)
        # print(hidden.permute(1, 0, 2).contiguous().shape, hidden.permute(1, 0, 2).contiguous().view(-1, self.max_seq_len*self.hidden_dim*2).shape)

        out = self.lstm2hidden(hidden.view(-1, 6*self.hidden_dim))  # batch_size x 4*hidden_dim
        # out = self.lstm2hidden(hidden.view(-1, self.max_seq_len*self.hidden_dim*2))  # batch_size x 4*hidden_dim
        
        out = torch.relu(out)
        # out = out * torch.sigmoid(0.1 * out) # функция активации Swish: x * sigmoid(b*x)
        out = self.dropout_linear(out)
        out = self.hidden2out(out)                                 # batch_size x 1
        out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp):
        """
        Classifies a batch of sequences.

        Inputs: inp
            - inp: batch_size x seq_len

        Returns: out
            - out: batch_size ([0,1] score)
        """

        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return out.view(-1)

    def batchBCELoss(self, inp, target):
        """
        Returns Binary Cross Entropy Loss for discriminator.

         Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size (binary 1/0)
        """

        loss_fn = nn.BCELoss()
        h, c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c)
        return loss_fn(out, target)

Объявление функций возвращающих данные

In [47]:
# функция генерации реалиных данных
def sampler_example(batch_size):
  x = data_file_train[np.random.randint(0, len(data_file_train), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

# функция генерации реалиных данных
def sampler_example_test(batch_size):
  x = data_file_test[np.random.randint(0, len(data_file_test), batch_size)]
  y = np.concatenate([x[:, 1:], np.zeros([batch_size, 1])+VOCAB_SIZE-2], axis=-1)
  return x, y

Функции обучения генератора

In [48]:
# функция обучения генератора на основе MLE
def train_generator_MLE(gen, gen_opt, real_samples_train, real_samples_test, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0

        # обучение
        for i in range(0, len(real_samples_train), BATCH_SIZE):
            inp_train, target_train = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                          gpu=CUDA)
            gen_opt.zero_grad()
            loss = gen.batchNLLLoss(inp_train, target_train)
            loss.backward()
            gen_opt.step()

            total_loss += loss.data.item()

            if (i / BATCH_SIZE) % ceil(
                            ceil(len(real_samples_train) / float(BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                print('.', end='')
                sys.stdout.flush()

        # each loss in a batch is loss per sample
        total_loss = total_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_train_NLL = %.4f' % total_loss, end='')

        # тестирование
        test_loss = 0

        for i in range(0, len(real_samples_test), BATCH_SIZE):
            inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                              gpu=CUDA)
            loss = gen.batchNLLLoss(inp_test, target_test)
            test_loss += loss.data.item()
        test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
        print(' average_test_NLL = %.4f' % test_loss)

def test_mle(gen, real_samples_train, real_samples_test):
  '''
  Тестирование генератора на обучающей и тестовой выборках.
  '''
  # тестирование на обучающей
  test_loss = 0
  for i in range(0, len(real_samples_train), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_train[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_train) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print('average_train_NLL = %.4f' % test_loss, end='')

  # тестирование на тестовой
  test_loss = 0
  for i in range(0, len(real_samples_test), BATCH_SIZE):
      inp_test, target_test = helpers.prepare_generator_batch(real_samples_test[i:i + BATCH_SIZE], start_letter=START_LETTER,
                                                        gpu=CUDA)
      loss = gen.batchNLLLoss(inp_test, target_test)
      test_loss += loss.data.item()
  test_loss = test_loss / ceil(len(real_samples_test) / float(BATCH_SIZE)) / MAX_SEQ_LEN
  print(' average_test_NLL = %.4f' % test_loss)

# функция обучения генератора на основе обучения с подкреплением RL
def train_generator_PG(gen, gen_opt, dis, num_batches): # !переименовать!
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """

    for batch in range(num_batches):
        s = gen.sample(BATCH_SIZE*4)        # 64 works best
        inp, target = helpers.prepare_generator_batch(s, start_letter=START_LETTER, gpu=CUDA)
        rewards = dis.batchClassify(target)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards)
        pg_loss.backward()
        gen_opt.step()
    print()

Функция обучения дискриминатора

In [49]:
# функция обучения дискриминатора
def train_discriminator(discriminator, dis_opt, real_data_samples, generator, d_steps, epochs):
    """
    Training the discriminator on real_data_samples (positive) and generated samples from generator (negative).
    Samples are drawn d_steps times, and the discriminator is trained for epochs epochs.
    """

    # generating a small validation set before training
    pos_val = real_data_samples[np.random.randint(0, len(real_data_samples), 500)] #sampler_example(250)
    neg_val = generator.sample(500)
    val_inp, val_target = helpers.prepare_discriminator_data(pos_val, neg_val, gpu=CUDA)

    for d_step in range(d_steps):
        s = helpers.batchwise_sample(generator, POS_NEG_SAMPLES, BATCH_SIZE)
        dis_inp, dis_target = helpers.prepare_discriminator_data(real_data_samples, s, gpu=CUDA)
        val_pred = discriminator.batchClassify(val_inp)
        print('ДО ОБУЧЕНИЯ: val_acc = %.4f' % (
            torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))
        for epoch in range(epochs):
            print('d-step %d epoch %d : ' % (d_step + 1, epoch + 1), end='')
            sys.stdout.flush()
            total_loss = 0
            total_acc = 0

            for i in range(0, 2 * POS_NEG_SAMPLES, BATCH_SIZE):
                inp, target = dis_inp[i:i + BATCH_SIZE], dis_target[i:i + BATCH_SIZE]
                dis_opt.zero_grad()
                out = discriminator.batchClassify(inp)
                loss_fn = nn.BCELoss()
                loss = loss_fn(out, target)
                loss.backward()
                dis_opt.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out>0.5)==(target>0.5)).data.item()

                if (i / BATCH_SIZE) % ceil(ceil(2 * POS_NEG_SAMPLES / float(
                        BATCH_SIZE)) / 10.) == 0:  # roughly every 10% of an epoch
                    print('.', end='')
                    sys.stdout.flush()

            total_loss /= ceil(2 * POS_NEG_SAMPLES / float(BATCH_SIZE))
            total_acc /= float(2 * POS_NEG_SAMPLES)

            val_pred = discriminator.batchClassify(val_inp)
            print(' average_loss = %.4f, train_acc = %.4f, val_acc = %.4f' % (
                total_loss, total_acc, torch.sum((val_pred>0.5)==(val_target>0.5)).data.item()/1000.))

Функция оценки качества генерации текста

In [50]:
# оценка качества по BLEU метрике
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from datetime import datetime
import nltk
import random
from scipy import stats

# функция оценки качества генерации текста по метрике BLEU
def BLEU(reference_sample, test_sample, print_iteration=100, flag_print=True):
  if flag_print:
    print("--- --- ---\nStart BLEU")
  pad = CLOSING_WORD
  #################################################
  reference = []
  for line in reference_sample:
    candidate = []
    for i in line:
      if i == pad:
        break
      candidate.append(i)

    reference.append(candidate)
  #################################################
  hypothesis_list_leakgan = []
  for line in test_sample:
    while line[-1] == str(pad):
      line.remove(str(pad))
    hypothesis_list_leakgan.append(line)
  #################################################
  random.shuffle(hypothesis_list_leakgan)
  #################################################

  smoothing_function = SmoothingFunction().method1

  mass_bleu = []
  for ngram in range(2,6):
      weight = tuple((1. / ngram for _ in range(ngram)))
      bleu_leakgan = []
      bleu_supervise = []
      bleu_base2 = []
      num = 0
      for h in hypothesis_list_leakgan:
          BLEUscore = nltk.translate.bleu_score.sentence_bleu(reference, h, weight, smoothing_function = smoothing_function)
          num += 1
          bleu_leakgan.append(BLEUscore)

          if num%print_iteration == 0 and flag_print:
            print(ngram, num, sum(bleu_leakgan)/len(bleu_leakgan))
          
      mass_bleu.append(1.0 * sum(bleu_leakgan) / len(bleu_leakgan))
      if flag_print:
        print('--- --- ---')
        print(len(weight), '-gram BLEU score : ', 1.0 * sum(bleu_leakgan) / len(bleu_leakgan), "\n")
  return mass_bleu

Функция сохранения моделей

In [51]:
# функция сохранения моделей (сериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer, name):
  state = {
      'default_parameters': {'VOCAB_SIZE': VOCAB_SIZE, 'MAX_SEQ_LEN': MAX_SEQ_LEN, 'GEN_EMBEDDING_DIM': GEN_EMBEDDING_DIM,
                             'GEN_HIDDEN_DIM': GEN_HIDDEN_DIM, 'DIS_EMBEDDING_DIM': DIS_EMBEDDING_DIM, 'DIS_HIDDEN_DIM': DIS_HIDDEN_DIM},
      'data_file_tensor_train': data_file_tensor_train,
      'gen_state_dict': gen.state_dict(),
      'dis_state_dict': dis.state_dict(),
      'gen_optimizer': gen_optimizer.state_dict(),
      'dis_optimizer': dis_optimizer.state_dict(),
  }
  torch.save(state, name)

Функция загрузки моделей

In [52]:
# функция загрузки моделей (десериализация: моделей генератора и дискриминатора, параметров по умолчанию, обучающих данных)
def load_models(name):
  if CUDA:
    device = torch.device('cuda')
  else:
    device = torch.device('cpu')

  print('state')
  state = torch.load(name, map_location=device)

  print('default_parameters')
  VOCAB_SIZE = state['default_parameters']['VOCAB_SIZE']
  MAX_SEQ_LEN = state['default_parameters']['MAX_SEQ_LEN']
  GEN_EMBEDDING_DIM = state['default_parameters']['GEN_EMBEDDING_DIM']
  GEN_HIDDEN_DIM = state['default_parameters']['GEN_HIDDEN_DIM']
  DIS_EMBEDDING_DIM = state['default_parameters']['DIS_EMBEDDING_DIM']
  DIS_HIDDEN_DIM = state['default_parameters']['DIS_HIDDEN_DIM']

  print('data_file_tensor_train')
  data_file_tensor_train = torch.tensor(state['data_file_tensor_train'])

  print('Generator')
  gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  gen.load_state_dict(state['gen_state_dict'])
  gen_optimizer = optim.Adam(gen.parameters(), lr=0.001)
  gen_optimizer.load_state_dict(state['gen_optimizer'])

  print('Discriminator')
  dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
  dis.load_state_dict(state['dis_state_dict'])
  dis_optimizer = optim.Adagrad(dis.parameters())
  dis_optimizer.load_state_dict(state['dis_optimizer'])

  print('CUDA')
  if CUDA:
    data_file_tensor_train = data_file_tensor_train.cuda()
    gen = gen.cuda()
    dis = dis.cuda()
  
  return [data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
          VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM]

Загрузка набора данных

In [53]:
# загрузка словаря
import pickle

vocab_file = FILE_PATHS['vocab']
word, vocab = pickle.load(open(vocab_file, 'rb'))

In [54]:
# загрузка обучающей выборки
f = open(FILE_PATHS['train'], 'r')
data_file_train = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_train.append(line)
data_file_train = np.array(data_file_train)[:, :MAX_SEQ_LEN]
print("Примеров в обучающей выборке: ", len(data_file_train))

Примеров в обучающей выборке:  70000


In [55]:
# загрузка тестовой выборки
f = open(FILE_PATHS['test'], 'r')
data_file_test = []
for line in f:
  line = line.replace('\n', '')
  line = line.split()
  for i in range(len(line)):
    line[i] = int(line[i])
  data_file_test.append(line)
data_file_test = np.array(data_file_test)[:, :MAX_SEQ_LEN]
print("Примеров в тестовой выборке: ", len(data_file_test))

Примеров в тестовой выборке:  5000


In [56]:
# примеры из обучающей выборки
print("Примеры из обучающей выборки")
samples = sampler_example(50)[0]
output_function = []
for samp in samples:
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)

for i, output in enumerate(output_function):
  print("#", i, "\tПример: ", output)    

Примеры из обучающей выборки
# 0 	Пример:  будь спокоен за счастье за нежность нашу и тепло 
# 1 	Пример:  но просить же я могу и желать такого  
# 2 	Пример:  ее тебе не удержать и не о чем говорить 
# 3 	Пример:  ты что то говоришь а мне тепло тепло  
# 4 	Пример:  настоящее добро в одном лишь боге больше его нет нигде
# 5 	Пример:  я здесь ты там но это не беда  
# 6 	Пример:  здесь однажды был я знал что снова буду  
# 7 	Пример:  но не все в этой жизни так просто  
# 8 	Пример:  а любви всё ж на всех не хватает  
# 9 	Пример:  густой исчезла лишь только шелест слышен был вдали  
# 10 	Пример:  но с надеждой к богу все к нему в пути
# 11 	Пример:  если не буду знать я где ты я знаю ты
# 12 	Пример:  но в чем оно счастье и где оно лето 
# 13 	Пример:  я б остался в том сне навсегда навсегда  
# 14 	Пример:  но что то есть что знаем только мы и звезды
# 15 	Пример:  а проснусь почему то я опять не с тобой 
# 16 	Пример:  дни летят за днями и за годом год  
# 17 	Пример:  я пью её а в

Оценка качества примеров из обучающей выборки на основе BLEU  (для сравнения с генератором)

In [57]:
%%time
# оценка качества на основе примеров из обучающей выборки (для сравнения с генератором)
print("Оценка примеров из обучающей выборки на основе BLEU")
BLEU(data_file_test.tolist(), data_file_train[:500].tolist(), print_iteration=100)

'''
BLEU примеров из обучающей выборки
--- --- ---
Start BLEU
2 100 0.5365504958995001
2 200 0.5426700180034696
2 300 0.5425210147596119
2 400 0.5438925313146432
2 500 0.5499117864293879
--- --- ---
2 -gram BLEU score :  0.5499117864293879
3 100 0.4252853298296578
3 200 0.4294441261529973
3 300 0.42938411299670276
3 400 0.432946142036555
3 500 0.436991427933691
--- --- ---
3 -gram BLEU score :  0.436991427933691
4 100 0.3118228221158724
4 200 0.31347352868277367
4 300 0.3133847969805193
4 400 0.3148005592801212
4 500 0.3173224414085976
--- --- ---
4 -gram BLEU score :  0.3173224414085976
5 100 0.20748007111715946
5 200 0.21057918753505803
5 300 0.20884027925072032
5 400 0.20986772415037908
5 500 0.21083373415979106
--- --- ---
5 -gram BLEU score :  0.21083373415979106
'''

Оценка примеров из обучающей выборки на основе BLEU
--- --- ---
Start BLEU
2 100 0.7557356298048126
2 200 0.7431149192926594
2 300 0.7406077524743515
2 400 0.7376227044540618
2 500 0.7379809479854459
--- --- ---
2 -gram BLEU score :  0.7379809479854459 

3 100 0.44528086673712786
3 200 0.4241709731237712
3 300 0.4263801126725085
3 400 0.4260298418663686
3 500 0.4254384872936007
--- --- ---
3 -gram BLEU score :  0.4254384872936007 

4 100 0.2437210033202152
4 200 0.22273400244271355
4 300 0.2249459514664226
4 400 0.22592203062778282
4 500 0.22426177276840145
--- --- ---
4 -gram BLEU score :  0.22426177276840145 

5 100 0.14527711133766108
5 200 0.13632915060821213
5 300 0.1382702406801177
5 400 0.13784283127701344
5 500 0.13625371525855653
--- --- ---
5 -gram BLEU score :  0.13625371525855653 

CPU times: user 5min 56s, sys: 115 ms, total: 5min 56s
Wall time: 5min 56s


Создание нейронных сетей генератора и дискриминатора

In [58]:
# объявление нейронных сетей генератора и дискриминатора, подготовка выборок данных для использования pytorch
gen = Generator(GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)
dis = Discriminator(DIS_EMBEDDING_DIM, DIS_HIDDEN_DIM, VOCAB_SIZE, MAX_SEQ_LEN, gpu=CUDA)

if CUDA:
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()
else:
  data_file_tensor_train = torch.tensor(data_file_train)
  data_file_tensor_test = torch.tensor(data_file_test)

gen_optimizer = optim.Adam(gen.parameters(), lr=0.001) #, lr=0.001
dis_optimizer = optim.Adagrad(dis.parameters())

Обучение генератора на основе MLE

In [59]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 8.5393 average_test_NLL = 8.5392
CPU times: user 641 ms, sys: 199 ms, total: 840 ms
Wall time: 840 ms


In [60]:
%%time
# обучение генератора на основе MLE / предобучение генератора
print('Запуск обучения генератора на основе MLE...')
gen_optimizer = optim.Adam(gen.parameters())#, lr=0.0002
train_generator_MLE(gen, gen_optimizer, data_file_tensor_train, data_file_tensor_test, MLE_TRAIN_EPOCHS) # MLE_TRAIN_EPOCHS

test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_mle.pytorch')
# epoch 1 : .......... average_train_NLL = 1.8447 average_test_NLL = 1.9937

Запуск обучения генератора на основе MLE...
epoch 1 : .......... average_train_NLL = 6.3798 average_test_NLL = 5.5416
epoch 2 : .......... average_train_NLL = 5.3908 average_test_NLL = 5.2943
epoch 3 : .......... average_train_NLL = 5.1978 average_test_NLL = 5.1441
epoch 4 : .......... average_train_NLL = 5.0625 average_test_NLL = 5.0328
epoch 5 : .......... average_train_NLL = 4.9601 average_test_NLL = 4.9514
epoch 6 : .......... average_train_NLL = 4.8811 average_test_NLL = 4.8908
epoch 7 : .......... average_train_NLL = 4.8178 average_test_NLL = 4.8431
epoch 8 : .......... average_train_NLL = 4.7642 average_test_NLL = 4.8038
epoch 9 : .......... average_train_NLL = 4.7174 average_test_NLL = 4.7706
epoch 10 : .......... average_train_NLL = 4.6757 average_test_NLL = 4.7418
epoch 11 : .......... average_train_NLL = 4.6378 average_test_NLL = 4.7162
epoch 12 : .......... average_train_NLL = 4.6029 average_test_NLL = 4.6933
epoch 13 : .......... average_train_NLL = 4.5706 average_test_NLL

In [61]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)

average_train_NLL = 4.1744 average_test_NLL = 4.5120
CPU times: user 655 ms, sys: 188 ms, total: 844 ms
Wall time: 845 ms


Генерация примеров текстов на основе MLE

In [62]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1
# 0 	Пример:  огня друг ответила за ним в кровать до сна                                                            	Оценка:  [0.6666666666666666, 0.381571414184444, 0.167844596251862, 0.10575371703212418]
# 1 	Пример:  так даёт лучами друг для толпы и такого нет                                                           	Оценка:  [0.3162277660168379, 0.10772173450159421, 0.06500593260343694, 0.04951447111434312]
# 2 	Пример:  а где то там ни минуты до края                                                                        	Оценка:  [0.9428090415820634, 0.7631428283688879, 0.5969491792019646, 0.2918230803979956]
# 3 	Пример:  и пусть теперь без страха в часа с е больше                                                           	Оценка:  [0.5773502691896257, 0.34668063717531744, 0.15619699684601282, 0.09984076352304341]
# 4 	Пример:  я в не в плену ты как это раньше сам                                                                

Оценка качества генерации текста после обучения с помощью MLE

In [63]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.7207746163633939
2 200 0.7102008706809321
2 300 0.714883145523868
2 400 0.7169106889930906
2 500 0.7191104366142823
--- --- ---
2 -gram BLEU score :  0.7191104366142823 

3 100 0.3778506843966224
3 200 0.37401724302890577
3 300 0.3747892007161154
3 400 0.3725004302247718
3 500 0.37587988607508577
--- --- ---
3 -gram BLEU score :  0.37587988607508577 

4 100 0.18629428171386284
4 200 0.18730292654987502
4 300 0.18779596434235124
4 400 0.1854892453738397
4 500 0.1861466902118776
--- --- ---
4 -gram BLEU score :  0.1861466902118776 

5 100 0.11410926993417572
5 200 0.11586304074775632
5 300 0.11600506518730475
5 400 0.11449217860642476
5 500 0.11463232316925863
--- --- ---
5 -gram BLEU score :  0.11463232316925863 

CPU times: user 5min 57s, sys: 121 ms, total: 5min 57s
Wall time: 5min 57s


Оценка с возведением в степень

In [64]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе MLE")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе MLE
Degree: 1.5
# 0 	Пример:  и я не знаю что я не могу                                                                             	Оценка:  [1.0, 1.0, 0.8091067115702212, 0.5898945623563899]
# 1 	Пример:  ты мне и не то и не сможешь                                                                           	Оценка:  [0.9428090415820634, 0.7631428283688879, 0.5969491792019646, 0.2918230803979956]
# 2 	Пример:  и в них нет там где нет боли                                                                          	Оценка:  [0.816496580927726, 0.43679023236814946, 0.18575057999133604, 0.11468692082056794]
# 3 	Пример:  и мы все хотим с тобой по кругу                                                                       	Оценка:  [0.8819171036881969, 0.7299198566479815, 0.4854917717073234, 0.24735189898367643]
# 4 	Пример:  я не знаю что в тебе я совсем придумал                                                                	Оценка:  [0.881917103688196

In [65]:
%%time
print("Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текста на основе BLEU, после обучения с помощью MLE
--- --- ---
Start BLEU
2 100 0.9110841130803087
2 200 0.9128592924385134
2 300 0.9146554172705765
2 400 0.9163526544163312
2 500 0.9131904285074142
--- --- ---
2 -gram BLEU score :  0.9131904285074142 

3 100 0.6632484121585793
3 200 0.6720061886944637
3 300 0.6777352343138564
3 400 0.6803502290187979
3 500 0.6792977554430498
--- --- ---
3 -gram BLEU score :  0.6792977554430498 

4 100 0.3799941689996252
4 200 0.3904176754214317
4 300 0.39871029395880414
4 400 0.4026617908579711
4 500 0.4001824956390621
--- --- ---
4 -gram BLEU score :  0.4001824956390621 

5 100 0.21590792026570313
5 200 0.22347565990667476
5 300 0.22796507131962357
5 400 0.23232880727386018
5 500 0.2305350903578656
--- --- ---
5 -gram BLEU score :  0.2305350903578656 

CPU times: user 5min 52s, sys: 114 ms, total: 5min 53s
Wall time: 5min 52s


Предварительное обучение дискриминатора

In [66]:
%%time
# предобучение дискриминатора
print('Запуск обучения дискриминатора...')
# dis_optimizer = optim.Adagrad(dis.parameters()) # , lr=0.0001
# dis_optimizer = optim.Adam(gen.parameters(), lr=0.001, weight_decay=1e-5)#0.001
# dis_optimizer = optim.Adadelta(gen.parameters())
# dis_optimizer = optim.Adagrad(dis.parameters())#, lr=0.0001)#, weight_decay=1e-5)
#, weight_decay=1e-5) # регуляризация
train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, DIS_TRAIN_ITERATIONS, DIS_TRAIN_EPOCHS)#25, 1 | (15, 3), (25, 1)

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_pretraining_dis.pytorch')

# ДО ОБУЧЕНИЯ: val_acc = 0.5230
# d-step 50 epoch 1 : .......... average_loss = 0.2859, train_acc = 0.9136, val_acc = 0.5240

Запуск обучения дискриминатора...
ДО ОБУЧЕНИЯ: val_acc = 0.5040
d-step 1 epoch 1 : .......... average_loss = 0.4592, train_acc = 0.8385, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 2 epoch 1 : .......... average_loss = 0.3759, train_acc = 0.8758, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 3 epoch 1 : .......... average_loss = 0.3779, train_acc = 0.8747, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 4 epoch 1 : .......... average_loss = 0.3778, train_acc = 0.8742, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 5 epoch 1 : .......... average_loss = 0.3681, train_acc = 0.8791, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 6 epoch 1 : .......... average_loss = 0.3765, train_acc = 0.8741, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 7 epoch 1 : .......... average_loss = 0.3772, train_acc = 0.8744, val_acc = 0.5000
ДО ОБУЧЕНИЯ: val_acc = 0.5000
d-step 8 epoch 1 : .......... average_loss = 0.3745, train_acc = 0.8753, val_acc = 0.5000
ДО ОБУ

Состязательное обучение. Обучение генератора на основе обучения с подкреплением

In [67]:
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.7516 average_test_NLL = 1.9832

average_train_NLL = 4.1744 average_test_NLL = 4.5120


In [68]:
%%time
# gen_optimizer = optim.Adagrad(gen.parameters(), lr=0.0005)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters())
# gen_optimizer = optim.Adam(gen.parameters(), lr=0.0001)#Adam #, lr=0.0005
# dis_optimizer = optim.Adagrad(dis.parameters(), lr=0.001)#, lr=0.002

# состязательное обучение генератора
print('\nStarting Adversarial Training...')

for epoch in range(ADV_TRAIN_EPOCHS):# ADV_TRAIN_EPOCHS
    print('\n--------\nEPOCH %d\n--------' % (epoch+1))
    # обучение генератора
    print('\nAdversarial Training Generator : ', end='')
    train_generator_PG(gen, gen_optimizer, dis, 1)#

    # тестирование nll
    print('\nTesting Generator : ', end='')
    test_mle(gen, data_file_tensor_train, data_file_tensor_test)

    # обучение дискриминатора
    print('\nAdversarial Training Discriminator : ')
    train_discriminator(dis, dis_optimizer, data_file_tensor_train, gen, 5, 1)#3, 1

# сохранение результата обучения
save_models(data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
            FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')
# epoch 3 : .......... average_train_NLL = 1.8005 average_test_NLL = 1.9824
# average_train_NLL = 1.7340 average_test_NLL = 2.0599


Starting Adversarial Training...

--------
EPOCH 1
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 4.1758 average_test_NLL = 4.5144

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val_acc = 0.7470
d-step 1 epoch 1 : .......... average_loss = 0.2180, train_acc = 0.9357, val_acc = 0.7370
ДО ОБУЧЕНИЯ: val_acc = 0.7450
d-step 2 epoch 1 : .......... average_loss = 0.2151, train_acc = 0.9364, val_acc = 0.7250
ДО ОБУЧЕНИЯ: val_acc = 0.7310
d-step 3 epoch 1 : .......... average_loss = 0.2179, train_acc = 0.9364, val_acc = 0.7500
ДО ОБУЧЕНИЯ: val_acc = 0.7540
d-step 4 epoch 1 : .......... average_loss = 0.2093, train_acc = 0.9377, val_acc = 0.7430
ДО ОБУЧЕНИЯ: val_acc = 0.7480
d-step 5 epoch 1 : .......... average_loss = 0.2116, train_acc = 0.9376, val_acc = 0.7600

--------
EPOCH 2
--------

Adversarial Training Generator : 

Testing Generator : average_train_NLL = 4.1776 average_test_NLL = 4.5166

Adversarial Training Discriminator : 
ДО ОБУЧЕНИЯ: val

Тестирование генератора на обучающей выборке после состязательного обучения.

In [69]:
%%time
test_mle(gen, data_file_tensor_train, data_file_tensor_test)
# average_train_NLL = 1.5186 average_test_NLL = 2.0730
# average_train_NLL = 1.4459 average_test_NLL = 1.9926

average_train_NLL = 4.2210 average_test_NLL = 4.5557
CPU times: user 626 ms, sys: 174 ms, total: 800 ms
Wall time: 821 ms


Генерация примеров текстов на основе SeqGAN

In [70]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN")
degree = 1
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе SeqGAN
Degree: 1
# 0 	Пример:  что мир у вас у каждого хранить точно                                                                 	Оценка:  [0.8819171036881969, 0.5793377741477417, 0.22957488466614331, 0.13586551826765378]
# 1 	Пример:  только русский история и сердце больше не хватало                                                     	Оценка:  [0.5477225575051661, 0.15536162529769298, 0.08555261858712451, 0.06168170862583606]
# 2 	Пример:  другой тебе твоя будет и всё что и дело                                                               	Оценка:  [0.816496580927726, 0.20274006651911342, 0.10445522730720386, 0.07236255500755572]
# 3 	Пример:  что к тебе а другой вид что он и нас                                                                  	Оценка:  [0.816496580927726, 0.43679023236814946, 0.18575057999133604, 0.11468692082056794]
# 4 	Пример:  но вот так что то есть и это                                                                 

Оценка качества генерации на основе SeqGAN

In [71]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
degree = 1
print("Degree:", degree)
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
Degree: 1
--- --- ---
Start BLEU
2 100 0.756327066391588
2 200 0.7551942931955969
2 300 0.7592751933861931
2 400 0.7548446136656939
2 500 0.7570414872078776
--- --- ---
2 -gram BLEU score :  0.7570414872078776 

3 100 0.4479584190894686
3 200 0.43899278585143875
3 300 0.44134262752863906
3 400 0.43509346796894255
3 500 0.43604133425918606
--- --- ---
3 -gram BLEU score :  0.43604133425918606 

4 100 0.225955953164321
4 200 0.2265542585882442
4 300 0.22464085190691174
4 400 0.2211950392895392
4 500 0.22264702121052837
--- --- ---
4 -gram BLEU score :  0.22264702121052837 

5 100 0.1355252852029599
5 200 0.13670329584898577
5 300 0.13403863938260258
5 400 0.13185758403281145
5 500 0.13229657890676344
--- --- ---
5 -gram BLEU score :  0.13229657890676344 

CPU times: user 5min 50s, sys: 127 ms, total: 5min 50s
Wall time: 5min 50s


Оценка с возведением в степень

In [72]:
# примеры сгенерированных текстов
print("Примеры генерируемых текстов на основе SeqGAN с возведением в степень")
degree = 1.5
print("Degree:", degree)
samples = gen.sample(50, degree=degree).cpu().detach().numpy()

output_function = []
for i, samp in enumerate(samples):
  line = [word[x] for x in samp]
  line = ' '.join(line)
  output_function.append(line)
  bleu = BLEU(data_file_test.tolist(), [samp], flag_print=False)
  print("#", i, "\tПример: ", line, ' '*(100-len(line)), '\tОценка: ', bleu)

Примеры генерируемых текстов на основе SeqGAN с возведением в степень
Degree: 1.5
# 0 	Пример:  и не знаю что я люблю тебя я                                                                          	Оценка:  [1.0, 1.0, 0.6147881529512643, 0.29877904991471826]
# 1 	Пример:  и в общем то что жизнь здесь и в конце                                                                	Оценка:  [0.9428090415820634, 0.6057068642773799, 0.42210681263745264, 0.22116053872466884]
# 2 	Пример:  а я не могу я жизнь одна                                                                              	Оценка:  [0.8366600265340756, 0.7047298732064892, 0.26591479484724945, 0.15281421358157987]
# 3 	Пример:  и я не могу понять не я не могу                                                                       	Оценка:  [0.9428090415820634, 0.7631428283688879, 0.5019724248795793, 0.25404674702328517]
# 4 	Пример:  что не хватает нам в жизни не так                                                                   

In [73]:
%%time
print("Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN")
# проверка качества обучения
BLEU(data_file_test.tolist(), gen.sample(500, degree=degree).cpu().detach().numpy().tolist(), print_iteration=100)

Оценка качества генерации текстов на основе BLEU, после обучения с помощью SeqGAN
--- --- ---
Start BLEU
2 100 0.9352075650568834
2 200 0.9291563408758435
2 300 0.9282022814564931
2 400 0.9259652759971634
2 500 0.925982021812217
--- --- ---
2 -gram BLEU score :  0.925982021812217 

3 100 0.7208715349142032
3 200 0.7159446183904312
3 300 0.7150713953092783
3 400 0.7162938237004007
3 500 0.7200018075100502
--- --- ---
3 -gram BLEU score :  0.7200018075100502 

4 100 0.44950025039298047
4 200 0.42683411306066005
4 300 0.43388785470455454
4 400 0.44002929573821903
4 500 0.44385890784792853
--- --- ---
4 -gram BLEU score :  0.44385890784792853 

5 100 0.25507868622560825
5 200 0.24027600196763424
5 300 0.24244563488438728
5 400 0.24952739811011998
5 500 0.2525968934982302
--- --- ---
5 -gram BLEU score :  0.2525968934982302 

CPU times: user 5min 47s, sys: 139 ms, total: 5min 47s
Wall time: 5min 47s


Загрузка сохраненной модели

In [74]:
# загрузка моделей
[data_file_tensor_train, gen, dis, gen_optimizer, dis_optimizer,
VOCAB_SIZE, MAX_SEQ_LEN, GEN_EMBEDDING_DIM, GEN_HIDDEN_DIM, DIS_EMBEDDING_DIM,
 DIS_HIDDEN_DIM] = load_models(FILE_PATHS['saved_models'] + r'/' + r'seqgan_adversarial_training.pytorch')#[seqgan_mle, seqgan_pretraining_dis, seqgan_adversarial_training]

if(CUDA):
  gen = gen.cuda()
  dis = dis.cuda()
  data_file_tensor_train = torch.tensor(data_file_tensor_train).cuda()
  data_file_tensor_test = torch.tensor(data_file_test).cuda()

state
default_parameters
data_file_tensor_train
Generator
Discriminator
CUDA


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


Оценка разнообразия текста

In [75]:
degree = 1.5

In [76]:
# плагиат из обучающей выборки
samples = gen.sample(1500, degree=degree).cpu().detach().numpy().tolist()
# samples = data_file_test.tolist()[:500]
train_samples = data_file_train.tolist()
n = 0
for i in range(len(samples)):
  if samples[i] in train_samples:
    n += 1
  if i%(len(samples)//10) == 0:
    print(i/len(samples)*100, "%")
print("Совпало сгенерированных примеров с обучающей выборкой: ", n, "из", len(samples))
print("Плагиат: ", n/len(samples)*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Совпало сгенерированных примеров с обучающей выборкой:  1 из 1500
Плагиат:  0.06666666666666667 %


In [77]:
# оригинальность примеров
N_samples = 1500
samples_1 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()
samples_2 = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

n = 0
for i in range(len(samples_1)):
  if samples_1[i] in samples_2:
    n += 1
  if i%(len(samples_1)//10) == 0:
    print(i/len(samples_1)*100, "%")

print("Оригинальность генерируемых примеров")
print("Совпало примеров из выборки 1 с примерами из выборки 2: ", n, "из", len(samples_1))
print("Плагиат: ", n/len(samples_1)*100, "%")
print("Оригинальность: ", (1-n/len(samples_1))*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Оригинальность генерируемых примеров
Совпало примеров из выборки 1 с примерами из выборки 2:  3 из 1500
Плагиат:  0.2 %
Оригинальность:  99.8 %


In [78]:
# оригинальность примеров подход 2
N_samples = 1500
samples = gen.sample(N_samples, degree=degree).cpu().detach().numpy().tolist()

i = 0
n = 0
while len(samples)>0:
  if samples[0] not in samples[1:]:
    n += 1
  if i%(N_samples//10) == 0:
    print(i/N_samples*100, "%")
  del(samples[0])
  i += 1

print("Оригинальность генерируемых примеров")
print("Сгенерировано оригинальных примеров: ", n, "из", N_samples)
print("Оригинальность: ", n/N_samples*100, "%")
print("Плагиат: ", (1-n/N_samples)*100, "%")

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
Оригинальность генерируемых примеров
Сгенерировано оригинальных примеров:  1497 из 1500
Оригинальность:  99.8 %
Плагиат:  0.20000000000000018 %
